In [ ]:
import pandas as pd
import re
import string
import cpi



/usr/local/lib/python3.6/dist-packages/cpi/__init__.py:46: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  warnings.warn(StaleDataWarning())


In [ ]:
pip install cpi

     |████████████████████████████████| 25.5MB 170kB/s 


In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 163 kB in 2s (92.3 kB/s)
Reading package lists... Done


In [ ]:
 # Start Spark session
from pyspark.sql import SparkSession
 from pyspark.ml.feature import RegexTokenizer, Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType


In [ ]:

spark = SparkSession.builder.appName("tokenizing").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
spark = SparkSession.builder.appName("sparkFunctions").getOrCreate()
from pyspark import SparkFiles
url ="https://raw.githubusercontent.com/timegnome/Project-ML-Movie-Maker/ben/data_joined_30k.csv"
spark.sparkContext.addFile(url)
df1 = pd.read_csv(url)

In [ ]:
df1

,Id,Id (Keywords.Csv),Imdb Id,Imdb Id (Movies Metadata.Csv),Keywords,Original Title,Original Title1,Primary Title,Release Date,Start Year,Status,Title,Budget,Revenue,Runtime,Vote Average,Vote Count
0,862,862,tt0114709,tt0114709,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",Toy Story,Toy Story,Toy Story,10/30/1995,1995,Released,Toy Story,30000000,373554033.0,81.0,7.7,5415.0
1,8844,8844,tt0113497,tt0113497,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",Jumanji,Jumanji,Jumanji,12/15/1995,1995,Released,Jumanji,65000000,262797249.0,104.0,6.9,2413.0
2,15602,15602,tt0113228,tt0113228,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",Grumpier Old Men,Grumpier Old Men,Grumpier Old Men,12/22/1995,1995,Released,Grumpier Old Men,0,0.0,101.0,6.5,92.0
3,31357,31357,tt0114885,tt0114885,"[{'id': 818, 'name': 'based on novel'}, {'id':...",Waiting to Exhale,Waiting to Exhale,Waiting to Exhale,12/22/1995,1995,Released,Waiting to Exhale,16000000,81452156.0,127.0,6.1,34.0
4,11862,11862,tt0113041,tt0113041,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",Father of the Bride Part II,Father of the Bride Part II,Father of the Bride Part II,2/10/1995,1995,Released,Father of the Bride Part II,0,76578911.0,106.0,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30516,289923,289923,tt0252966,tt0252966,"[{'id': 616, 'name': 'witch'}, {'id': 2035, 'n...",The Burkittsville 7,The Burkittsville 7,The Burkittsville 7,10/3/2000,2000,Released,The Burkittsville 7,0,0.0,30.0,7.0,1.0
30517,222848,222848,tt0112613,tt0112613,[],Caged Heat 3000,Caged Heat 3000,Caged Heat 3000,1/1/1995,1995,Released,Caged Heat 3000,0,0.0,85.0,3.5,1.0
30518,30840,30840,tt0102797,tt0102797,[],Robin Hood,Robin Hood,Robin Hood,5/13/1991,1991,Released,Robin Hood,0,0.0,104.0,5.7,26.0
30519,67758,67758,tt0303758,tt0303758,[],Betrayal,Betrayal,Betrayal,8/1/2003,2003,Released,Betrayal,0,0.0,90.0,3.8,6.0


In [ ]:
df1=df1.drop(columns=['Id','Imdb Id (Movies Metadata.Csv)',])

In [ ]:
txt=df1['Keywords'][0]


In [ ]:
txt

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [ ]:
x = re.sub("{'id': [0-9]+, 'name': '","",txt)
x = re.sub("\[|'}\]","",x)
x = re.sub("'}, ",",",x)
print (x)

jealousy,toy,boy,friendship,friends,rivalry,boy next door,new toy,toy comes to life


In [ ]:
df1['Keywords'].head()

0    [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1    [{'id': 10090, 'name': 'board game'}, {'id': 1...
2    [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3    [{'id': 818, 'name': 'based on novel'}, {'id':...
4    [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
Name: Keywords, dtype: object

In [ ]:

new_keywords = [] 
for i in df1["Keywords"]: 
       x = re.sub("{'id': [0-9]+, 'name': '","",i)
       x = re.sub("\[|'}\]","",x)
       x = re.sub("'}, ",",",x)
       x=x.split(',')
       new_keywords.append(x)
       
df1["New Key"] = new_keywords





In [ ]:
df1['New Key'][3]

['based on novel',
 'interracial relationship',
 'single mother',
 'divorce',
 'chick flick']

In [ ]:
main_df=df1.drop(columns=['Keywords','Original Title','Original Title1','Primary Title', 'Status'])

,Id,Id (Keywords.Csv),Imdb Id,Imdb Id (Movies Metadata.Csv),Keywords,Original Title,Original Title1,Primary Title,Release Date,Start Year,Status,Title,Budget,Revenue,Runtime,Vote Average,Vote Count
0,862,862,tt0114709,tt0114709,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",Toy Story,Toy Story,Toy Story,10/30/1995,1995,Released,Toy Story,30000000,373554033.0,81.0,7.7,5415.0
1,8844,8844,tt0113497,tt0113497,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",Jumanji,Jumanji,Jumanji,12/15/1995,1995,Released,Jumanji,65000000,262797249.0,104.0,6.9,2413.0
2,15602,15602,tt0113228,tt0113228,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",Grumpier Old Men,Grumpier Old Men,Grumpier Old Men,12/22/1995,1995,Released,Grumpier Old Men,0,0.0,101.0,6.5,92.0
3,31357,31357,tt0114885,tt0114885,"[{'id': 818, 'name': 'based on novel'}, {'id':...",Waiting to Exhale,Waiting to Exhale,Waiting to Exhale,12/22/1995,1995,Released,Waiting to Exhale,16000000,81452156.0,127.0,6.1,34.0
4,11862,11862,tt0113041,tt0113041,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",Father of the Bride Part II,Father of the Bride Part II,Father of the Bride Part II,2/10/1995,1995,Released,Father of the Bride Part II,0,76578911.0,106.0,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30516,289923,289923,tt0252966,tt0252966,"[{'id': 616, 'name': 'witch'}, {'id': 2035, 'n...",The Burkittsville 7,The Burkittsville 7,The Burkittsville 7,10/3/2000,2000,Released,The Burkittsville 7,0,0.0,30.0,7.0,1.0
30517,222848,222848,tt0112613,tt0112613,[],Caged Heat 3000,Caged Heat 3000,Caged Heat 3000,1/1/1995,1995,Released,Caged Heat 3000,0,0.0,85.0,3.5,1.0
30518,30840,30840,tt0102797,tt0102797,[],Robin Hood,Robin Hood,Robin Hood,5/13/1991,1991,Released,Robin Hood,0,0.0,104.0,5.7,26.0
30519,67758,67758,tt0303758,tt0303758,[],Betrayal,Betrayal,Betrayal,8/1/2003,2003,Released,Betrayal,0,0.0,90.0,3.8,6.0
